In [1]:
from app_naver_news_scrap import *

In [2]:
scrap_targets = load_scrap_target()

In [3]:
scrap_targets

,political_party,candidate_nm,candidate_title,last_dttm
0,더불어민주당,채현일,구청장,None
1,개혁신당,허은아,의원,None
2,국민의힘,김영주,의원,None


In [4]:
row = scrap_targets.iloc[0]

In [5]:
row

political_party    더불어민주당
candidate_nm          채현일
candidate_title       구청장
last_dttm            None
Name: 0, dtype: object

In [6]:

candidate_nm = row['candidate_nm']
last_dttm = row['last_dttm']

if pd.isnull(last_dttm):
    last_dttm = None

news_list = naver_url_load(
    candidate_nm,
    target_date=last_dttm,
    size=1000,
    sort='date'
)

In [28]:
news = news_list[21]
news

{'title': "[총선 D-30] 서울 48석 빅매치…與 '수도 탈환'·野 '총력 사수'",
 'originallink': 'https://www.yna.co.kr/view/AKR20240305164000001',
 'link': 'https://n.news.naver.com/mnews/article/001/0014550102?sid=165',
 'description': '영등포갑은 민주당을 탈당한 뒤 국민의힘 소속으로 4선에 도전하는 국회부의장 출신의 김영주 의원, 민주당 <b>채현일</b> 전 영등포구청장이 격돌한다. 여기에다 국민의힘 비례대표 의원직을 던지고 개혁신당에 합류한 허은아... ',
 'pubDate': 'Fri, 08 Mar 2024 06:01:00 +0900'}

In [29]:
news['originallink']

'https://www.yna.co.kr/view/AKR20240305164000001'

In [46]:
url = 'https://vop.co.kr/A00001648393.html'

In [47]:
import requests
import urllib3
import ssl


class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)


def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

In [49]:

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Whale/3.20.182.14 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
}
res = get_legacy_session().get(url, headers=headers)

SSLError: HTTPSConnectionPool(host='vop.co.kr', port=443): Max retries exceeded with url: /A00001648393.html (Caused by SSLError(SSLError(1, '[SSL: DH_KEY_TOO_SMALL] dh key too small (_ssl.c:1006)')))

In [40]:
res.text

'<!doctype html>\n<html lang="ko" class="ko">\n<head itemscope>\n<meta charset="utf-8">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="viewport" content="width=device-width,initial-scale=1.0,maximum-scale=1.0,minimum-scale=1.0,user-scalable=no">\n<meta name="robots" content="max-image-preview:large, max-video-preview:0">\n<link rel="canonical" id="canonical" href="https://www.yna.co.kr/view/AKR20240305164000001" />\n<meta property="og:url" content="https://www.yna.co.kr/view/AKR20240305164000001" />\n<meta name="twitter:url" content="https://www.yna.co.kr/view/AKR20240305164000001" />\n<meta itemprop="url" content="https://www.yna.co.kr/view/AKR20240305164000001" />\n<link rel="amphtml" id="ynaMobileUrl" href="https://m.yna.co.kr/amp/view/AKR20240305164000001">\n<meta property="fb:app_id" content="1013291918759197" />\n<meta property="fb:pages" content="130229496988833" />\n<meta property="og:type" content="article" />\n<meta name="resource-type" content="article" 

In [25]:
result = scrap_news(news)
result

SSLError: HTTPSConnectionPool(host='www.yna.co.kr', port=443): Max retries exceeded with url: /view/AKR20240305164000001 (Caused by SSLError(SSLError(1, '[SSL: UNSAFE_LEGACY_RENEGOTIATION_DISABLED] unsafe legacy renegotiation disabled (_ssl.c:1006)')))

In [9]:
result['kywd'] = candidate_nm

In [10]:
result

{'title': '‘2찍 발언’ 이재명 “대단히 부적절, 정중히 사과”',
 'origin_url': 'http://www.segye.com/newsView/20240309505535?OutUrl=naver',
 'naver_url': 'https://n.news.naver.com/mnews/article/022/0003912192?sid=100',
 'summary': '이 대표는 지난 5일 서울 영등포갑 <b>채현일</b> 후보 지원유세에서 국민의힘에 입당한 김영주(4선·서울 영등포갑) 국회부의장이 영등포갑에 전략 공천된 것을 두고 “우리가 (채 후보를) 단수추천하지 않고 (김 부의장과)... ',
 'publish_date': '20240309',
 'publish_time': '153900',
 'media_nm': 'segye',
 'text': '\n\n李 ‘사이다’ 발언과 관련 사과는 매우 이례적\n이재명 더불어민주당 대표가 현장 유세 중 말한 ‘2찍’ 표현이 논란이 되자 결국 사과했다.\n이 대표가 자신 특유의 ‘사이다’ 발언과 관련해 사과한 것은 매우 이례적이다.\n이 대표는 9일 자신의 소셜미디어(SNS)에 “어제 지역구에서 사용했던 ‘2찍’ 표현에 죄송한 마음이 앞선다. 저의 발언은 대단히 부적절했다”며 이 같이 말했다.\n이어 이 대표는 “상대당을 지지하는 국민도, 민주당을 지지하는 국민도 모두 똑같은 주권자이고 이 나라의 주인” 이라며 “국정 운영의 무거운 책임을 맡고도 이 나라 주인의 의사를 무시하고 오만 속에 국정을 손놓고 있는 윤석열 정권을 심판해달라”고 말했다. 그러면서 “더 낮은 자세로 더 국민과 가까이 국민의 뜻을 온전히 섬기는 정치 하겠다”고 덧붙였다.\n이 대표의 2찍 표현은 지난 8일 인천 계양을 방문한 현장 유세 유튜브 영상에서 송출됐다. 해당 영상에는 이 대표가 식당에 식사하던 시민들과 인사를 나누며 “1번 이재명”이라고 말하는 장면이 담겼다. 이에 시민이 “네”라고 대답하자 이 대표는 웃으면서 말을 덧붙였는데 “설

In [11]:
insert_data(result)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "kywd" of relation "news_scrap_collect" does not exist
LINE 4:     kywd
            ^

[SQL: 
INSERT INTO news_scrap_collect (
    pvdr_cd,
    kywd
) VALUES (
    '001',
    '채현일'
) RETURNING collect_tno
;
]
(Background on this error at: https://sqlalche.me/e/20/f405)

{'title': '서울·인천 지지율 떼고 붙어도…與 한동훈 ‘호재’, 野 이재명 ‘비상’',
 'originallink': 'https://www.ekn.kr/web/view.php?key=20240308024161498',
 'link': 'https://www.ekn.kr/web/view.php?key=20240308024161498',
 'description': '이 지역 경쟁자는 민주당 친명계인 <b>채현일</b> 전 영등포구청장이다. 이미 전 국회부의장과 전 구청장이라는 체급차가 선명할 뿐더러, 영등포갑은 17~18대 총선 연속 보수계열 후보가 승리했지만 김영주 의원이 탈환해 내리... ',
 'pubDate': 'Fri, 08 Mar 2024 10:00:00 +0900'}